# ```uavsar_pytools``` Tutorial

In this notebook, we will display the basic functionality of using ```asf_search``` to find UAVSAR data products, and ```uavsar_pytools``` convert those products in analysis ready GeoTiifs. UAVSAR files come in a flat binary format, which are not ready for analysis. This package allows for the systematic conversion of binary files into GeoTiffs using information from the UAVSAR annoation file.

In [1]:
import asf_search as asf
from uavsar_pytools.UavsarImage import UavsarImage #single image class
from uavsar_pytools.uavsar_tools import grd_tiff_convert #single image class
from uavsar_pytools.UavsarScene import UavsarScene # binary convert to .tiffs
import os  # for chdir, getcwd, path.basename, path.exists
from datetime import datetime
import progressbar
from tqdm import tqdm
import logging
import glob
import requests
import zipfile
#from uavsar_pytools.convert import convert_image
#from uavsar_pytools.download import download_image

# for map to generate wkt
from folium import Map
from folium.plugins import Draw
import geopandas as gpd

# plotting libraries
import numpy as np #for log10, mean, percentile, power
import rasterio as rio
from rasterio.plot import show # plotting raster data
from rasterio.plot import show_hist #histograms of raster data
import matplotlib.pyplot as plt 

### Finding UAVSAR data

UAVSAR data is located in multiple places online 

### Create Folium map to generate at WKT file

The ```asf_search``` package uses the Well-known text or [WKT](https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry) files to indentify areas of interest (AOI) to search thier database. The [Folium](https://python-visualization.github.io/folium/#:~:text=folium%20makes%20it%20easy%20to,as%20markers%20on%20the%20map.) map created below allows users to generate a WKT by drawing a polygon directly on the map and exporting it to thier local machine. instead of having to write it out by hand. 

Another tool for generating WKT files is the [ASF Vertex](https://search.asf.alaska.edu/#/) platform.

The [UAVSAR Data Search](https://uavsar.jpl.nasa.gov/cgi-bin/data.pl) shows the extent of all UAVSAR swaths. This is a good place to start to base your area of interest (AOI) search on using the Folium map.

In [2]:
wkt = gpd.read_file('/Users/jacktarricone/Downloads/asf_wkt (12).geojson').to_wkt().loc[0, 'geometry']
print(wkt)

POLYGON ((-120.227509 39.403305, -120.227509 39.422403, -120.196609 39.422403, -120.196609 39.403305, -120.227509 39.403305))


In [3]:
# define search parameters for sierra flight line
start_date = datetime.strptime('2020-02-14 11:00:00', '%Y-%m-%d %H:%M:%S') 
end_date = datetime.strptime('2020-02-15 11:00:00', '%Y-%m-%d %H:%M:%S') 

## Seach for data
-use asf search, talk about all the parameters that can be used for search
-show different types of processing, state that GRD is important
-

In [4]:
# search for data
results = asf.search(platform = 'UAVSAR',
                     processingLevel = (['INTERFEROMETRY_GRD']),
                     intersectsWith = wkt,
                     start = start_date,
                     end = end_date)

print(f'{len(results)} product(s) found')

1 product(s) found


In [7]:
# describe this whole "class" thing...
type(first)

asf_search.ASFProduct.ASFProduct

In [5]:
# pull out three search results and test print the first one
first = results[0]
print(first.properties)

{'beamModeType': 'RPI', 'browse': ['https://datapool.asf.alaska.edu/BROWSE/UA/donner_03904_20002-003_20009-026_0019d_s01_L090HH_01.cor.png', 'https://datapool.asf.alaska.edu/BROWSE/UA/donner_03904_20002-003_20009-026_0019d_s01_L090HH_01.hgt.png'], 'bytes': '524731624', 'faradayRotation': None, 'fileID': 'UA_donner_03904_20002-003_20009-026_0019d_s01_L090_01-INTERFEROMETRY_GRD', 'fileName': 'donner_03904_20002-003_20009-026_0019d_s01_L090_01_int_grd.zip', 'flightDirection': None, 'frameNumber': '786', 'granuleType': 'UAVSAR_INSAR_SCENE', 'groupID': 'UA_donner_03904_20002-003_20009-026_0019d_s01_L090_01', 'insarStackId': '-1', 'md5sum': '2360c66931e4854785d689e3b286d23a', 'offNadirAngle': None, 'orbit': '-1', 'pathNumber': '03904', 'platform': 'UAVSAR', 'pointingAngle': None, 'polarization': 'HH', 'processingDate': '2021-11-10T01:08:27Z', 'processingLevel': 'INTERFEROMETRY_GRD', 'sceneName': 'UA_donner_03904_20002-003_20009-026_0019d_s01_L090_01', 'sensor': 'UAVSAR', 'startTime': '2020-0

## Define URLs for Download


In [9]:
# define the URLs and print, decribe this functionality is within the ASFproduct class
first_url = first.properties['url']
print(first_url)

https://datapool.asf.alaska.edu/INTERFEROMETRY_GRD/UA/donner_03904_20002-003_20009-026_0019d_s01_L090_01_int_grd.zip


In [10]:
# define outpout dir and list files
os.chdir('/Users/jacktarricone/Desktop/zach_test/sagehen/')
output_dir = '/Users/jacktarricone/Desktop/zach_test/sagehen/'

In [14]:
# define image to download
# download_image(url = first_url, output_dir = output_dir)
scene = UavsarScene(url = first_url, work_dir='./data/imgs/')
scene.url_to_tiffs()

Unzipping: 100%|██████████████████████████████| 5/5 [00:09<00:00,  1.93s/file]


In [ ]:
# define zip insar file we just downloaded
zipped_data = glob.glob("*.zip") # print contents of output directory
print(zipped_data[0])

In [ ]:
# unzip!
with zipfile.ZipFile(zipped_data[0], "r") as zip_ref:
    zip_ref.printdir()
    print('Extracting all the files now...')
    zip_ref.extractall()
    print("done")

# Convert Images to GeoTiffs
Once all our files are unzipped, we can use ```convert_to_tiff``` function to convert the binary images into geocoded rasters in WSG-84. It does this by pulling out information from the annotation file.

In [ ]:
# define zip insar file we just downloaded
grd_data = glob.glob("*.grd") # print contents of output directory
ann_file = glob.glob("*.ann")
print(grd_data)
print(ann_file)

In [ ]:
# import function to conver to .tiffs
from uavsar_pytools.tiff_conversion import grd_tiff_convert

In [ ]:
# convert all bin images from zip to .tif
#img1.convert_to_tiff(grd_data, overwrite = True)
for file in grd_data:
    grd_tiff_convert(in_fp = file,
                     ann_fp = ann_file[0],
                     out_dir = output_dir,
                     overwrite = False)

In [ ]:
# search for unw and ann files for download test
unw = glob.glob(os.path.abspath("*cor.grd.tiff"))
print(unw[0])

In [ ]:
unw_rast = rio.open(unw[0])
meta_data = unw_rast.profile
print(meta_data)

In [ ]:
show_hist(unw_rast, bins = 50)

In [ ]:
fig, ax = plt.subplots(figsize = (10,7))
ax.set_title("Sagehen Coherence",fontsize = 16);
show((unw_rast, 1), cmap = 'viridis'); # info from histogram

In [ ]:
# lowman_wkt = gpd.read_file('/Users/jacktarricone/Downloads/asf_wkt (8).geojson').to_wkt().loc[0, 'geometry']
# print(lowman_wkt)
# lowman_wkt = 'POLYGON ((-116.164284 43.670603, -116.164284 43.68277, -116.135445 43.68277, -116.135445 43.670603, -116.164284 43.670603))'